In [4]:

# coding: utf-8

# In[118]:

import numpy as np
import h5py
import scipy.io
np.random.seed(1337) # for reproducibility
import matplotlib.pyplot as plt

from keras.preprocessing import sequence
from keras.optimizers import RMSprop, Adam
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Convolution2D, MaxPooling1D, AtrousConvolution1D
from keras.regularizers import l2, activity_l1
from keras.constraints import maxnorm
from keras.layers.recurrent import LSTM, GRU
from keras.callbacks import ModelCheckpoint, EarlyStopping
# from seya.layers.recurrent import Bidirectional # after git clone need to python setup.py install again
# from keras.utils.layer_utils import print_layer_shapes



print 'loading data'
# trainmat = h5py.File('/Users/jx/Downloads/deepsea_train/train.mat')
validmat = scipy.io.loadmat('/Users/jx/Documents/rbznn/valid_enriched_rbzcontinuous_nomemorize_wstruct.mat')
# testmat = scipy.io.loadmat('/Users/jx/Documents/rbznn/test_enriched_rbzcontinuous_nomemorize.mat')
trainmat = scipy.io.loadmat('/Users/jx/Documents/rbznn/train_enriched_rbzcontinuous_nomemorize_wstruct.mat')



X_train = np.transpose(np.array(trainmat['tr'][0][0][0]),axes=(0,2,3,1))
y_train = np.array(trainmat['tr'][0][0][1]).squeeze()
# y_train = y_train.reshape((-1, 1))
# y_train.squeeze()
print X_train.shape
print y_train.shape



lr = 1e-6#learning rate
reg = 1e-3
print 'building model'
nb_filters=32
model = Sequential()
# model.add(LSTM(32,  W_regularizer=l2(reg),return_sequences=True, input_shape=(4, 113)))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(LSTM(64,  W_regularizer=l2(reg),return_sequences=True)) # return sequences is needed for stacking
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(LSTM(128,  W_regularizer=l2(reg)))
# model.add(Activation('relu'))
# model.add(Dropout(0.2))
# model.add(Dense(30))
# model.add(Flatten( input_shape=(4,113)))

# model.add(Dense(10))
# model.add(Dense(1))

model.add(Convolution2D(32, 2,29, border_mode='same', W_regularizer=l2(reg), input_shape=(X_train.shape[1], X_train.shape[2], X_train.shape[3]))) # adding conv layer collapses output
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Convolution2D(32, 2,4, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Convolution2D(32, 2,4, border_mode='same', W_regularizer=l2(reg))) # adding conv layer collapses output
model.add(Activation('relu'))
model.add(Dropout(0.2))





model.add(Flatten())


# model.add(Dense(30))
model.add(Dense(1))
adam = Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=1e-08)


print 'compiling model'
model.compile(loss='mse', optimizer='adam', metrics=["mse"])



checkpointer = ModelCheckpoint(filepath="bestmodel.hdf5", verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

X_valid=np.transpose(validmat['tr'][0][0][0],axes=(0,2,3,1))
y_valid=np.array(validmat['tr'][0][0][1]).squeeze()


model.fit(X_train, y_train, batch_size=256, nb_epoch=500, shuffle=True, validation_data=(X_valid, y_valid),callbacks=[checkpointer,earlystopper])





loading data
(48345, 2, 4, 113)
(48345,)
building model
compiling model
Train on 48345 samples, validate on 2553 samples
Epoch 1/500
48345/48345 [==============================] - 111s - loss: 1.2779 - mean_squared_error: 1.2151 - val_loss: 1.2995 - val_mean_squared_error: 1.2995
Epoch 2/500
48345/48345 [==============================] - 114s - loss: 1.2056 - mean_squared_error: 1.1518 - val_loss: 1.2078 - val_mean_squared_error: 1.2078
Epoch 3/500
48345/48345 [==============================] - 114s - loss: 1.1620 - mean_squared_error: 1.1116 - val_loss: 1.1838 - val_mean_squared_error: 1.1838
Epoch 4/500
48345/48345 [==============================] - 114s - loss: 1.1204 - mean_squared_error: 1.0691 - val_loss: 1.1951 - val_mean_squared_error: 1.1951
Epoch 5/500
48345/48345 [==============================] - 114s - loss: 1.0769 - mean_squared_error: 1.0216 - val_loss: 1.1995 - val_mean_squared_error: 1.1995
Epoch 6/500
48345/48345 [==============================] - 114s - loss: 1.0249 

In [76]:
%matplotlib

out=model.predict(X_train, batch_size=512,verbose=1)
plt.plot(y_train,out,'ro')

model.summary()


Using matplotlib backend: MacOSX
46080/46658 [============================>.] - ETA: 0s____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
flatten_38 (Flatten)             (None, 452)           0           flatten_input_6[0][0]            
____________________________________________________________________________________________________
dense_64 (Dense)                 (None, 10)            4530        flatten_38[0][0]                 
____________________________________________________________________________________________________
dense_65 (Dense)                 (None, 1)             11          dense_64[0][0]                   
____________________________________________________________________________________________________
dense_66 (Dense)                 (None, 1)             2           dense_65[0][0]                   
Tota

In [7]:
%matplotlib

out=model.predict(X_valid, batch_size=512,verbose=1)
plt.plot(y_valid,out,'ro')

model.summary()

Using matplotlib backend: MacOSX
2553/2553 [==============================] - 2s     
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
convolution2d_4 (Convolution2D)  (None, 32, 4, 113)    3744        convolution2d_input_2[0][0]      
____________________________________________________________________________________________________
activation_4 (Activation)        (None, 32, 4, 113)    0           convolution2d_4[0][0]            
____________________________________________________________________________________________________
dropout_4 (Dropout)              (None, 32, 4, 113)    0           activation_4[0][0]               
____________________________________________________________________________________________________
convolution2d_5 (Convolution2D)  (None, 32, 4, 113)    8224        dropout_4[0][0]                  
_____